In [1]:
# Importing Libarries

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

In [2]:
# loading model

model = load_model('asl_better_model.h5')

In [3]:
from skimage.transform import resize
def detect(frame):
    img = resize(frame, (64, 64, 3))
    img = np.expand_dims(img, axis = 0)
    if np.max(img) > 1:
        img = img/255.0
    prediction = model.predict(img)
    print(prediction)
    return prediction

In [4]:
import json

with open("class_indices.json", "r") as f:
    class_indices = json.load(f)

# Create a reverse lookup list
index = [letter for letter, _ in sorted(class_indices.items(), key=lambda item: item[1])]



In [5]:
frame = cv2.imread(r"C:\Users\thula\Documents\Mini_Project\Dataset\training_set\D\D16.jpg")
data = detect(frame)
predicted_class = index[np.argmax(data)]
confidence = np.max(data)
print(f"Predicted: {predicted_class}, Confidence: {confidence:.2f}")



1/1 [==============================] - 0s 233ms/step
[[1.8491182e-05 4.4501899e-03 8.3702682e-03 5.2320713e-01 6.5989872e-05
  6.0846662e-05 2.5506641e-04 4.4349451e-07 2.8218259e-04 7.2632315e-06
  1.5661784e-04 3.0331361e-01 1.5897957e-04 2.0900837e-04 8.8861823e-02
  4.0183833e-05 1.8252711e-05 5.2225944e-02 1.0009575e-04 6.4793671e-04
  1.1054519e-02 2.8884845e-04 1.4881233e-06 6.1982945e-03 1.0306788e-06
  5.4869106e-06]]
Predicted: D, Confidence: 0.52


# OpenCV

In [6]:
# Importing Libraries



In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from collections import deque

# Load the trained ASL model
model = load_model("asl_better_model.h5", compile=False)
model.summary()

# Class labels
labels = [chr(i) for i in range(65, 91)]  # 'A' to 'Z'
prediction_buffer = deque(maxlen=5)

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    frame = cv2.flip(frame, 1)  # Mirror correction

    # Region of Interest (ROI)
    x1, y1, x2, y2 = 800, 100, 1100, 400
    roi = frame[y1:y2, x1:x2]
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Preprocess the ROI
    roi_resized = cv2.resize(roi, (64, 64))
    roi_rgb = cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB)
    roi_array = img_to_array(roi_rgb) / 255.0
    roi_array = np.expand_dims(roi_array, axis=0)

    # Prediction
    #prediction = model.predict(roi_array, verbose=0)
    # predicted_class = labels[np.argmax(prediction)]
    #confidence = np.max(prediction) * 100
    prediction = model.predict(roi_array, verbose=0)
    predicted_class = labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

# Use only high-confidence predictions
    if confidence > 70:
        prediction_buffer.append(predicted_class)

    # Check if most recent predictions are consistent
        if len(prediction_buffer) == prediction_buffer.maxlen:
            if prediction_buffer.count(predicted_class) > 3:
                display_text = f"Predicted: {predicted_class}"
            else:
                display_text = "Waiting for consistent prediction..."
        else:
            display_text = "Analyzing..."
    else:
        display_text = "Waiting for clearer sign..."


    # Display result
    cv2.putText(frame, f"Predicted: {predicted_class}", (50, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
    cv2.putText(frame, f"Confidence: {confidence:.2f}%", (50, 140),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    # Show frame
    cv2.imshow('ASL Real-Time Recognition', frame)

    # Check for 'q' key press
    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        print("Exiting...")
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 256)        0